## 练习集 1

### 选择题 
**问题1.** 以下哪项不是优化模型的关键组成部分？
- 决策变量
- 参数
- 目标函数
- `超参数`

**问题2.** 数学优化可以严格替代几乎所有机器学习问题
- 正确
- `错误`

**问题3.** 机器学习可以严格替代几乎所有数学优化问题
- 正确
- `错误`

**问题4.** 以下哪项是数学优化中的决策变量类型？
- `二元变量`
- 复数变量
- 特征变量
- 互补变量

**问题5.** 为了求解混合整数规划，求解器可能需要求解以下数量的线性规划：
- 无
- 决策变量的数量
- 总约束条件的数量
- `可能是指数级别的数量`

**问题6.** 在尝试`处方性`分析（如数学优化）之前，您或您的数据分析团队必须在`预测性`分析方面发展专业知识。
- 正确
- `错误`

**问题7.** 线性规划（LP）的`可行区域`将比其对应的混合整数规划（MIP）有____点，假设这两个模型除了LP只有连续变量而MIP包含整数变量之外完全相同。
- `更多`
- 更少
- 总是完全相同

**问题8.** 除非特别指定，使用`addVars()`时的默认变量类型是**连续型**。
- `正确`
- 错误

设 $J = \{\texttt{Apple, Banana, Coconut, Dragonfruit, Elderberry, Fig, Gooseberry}\}$ 且 $T = \{1, 2, 3, 4\}$ 

**问题9-a.** 使用`addVars(J,...)`和`addVars(range(8),...)`添加决策变量将向模型中添加*相同*数量的变量
- 正确
- `错误`

**问题9-b.** 使用上述集合，使用`addVars(J, T,...)`和`addVars(range(28),...)`添加决策变量将向模型中添加*相同*数量的变量
- `正确`
- 错误

### 建模和编码 
下面是一个单元格中完整原始模型的代码，如果您想用它来帮助完成这些练习。

In [ ]:
%pip install gurobipy

import pandas as pd
import gurobipy as gp
from gurobipy import GRB

production = ['Baltimore','Cleveland','Little Rock','Birmingham','Charleston']
distribution = ['Columbia','Indianapolis','Lexington','Nashville','Richmond','St. Louis']

path = 'https://raw.githubusercontent.com/Gurobi/modeling-examples/master/optimization101/Modeling_Session_1/'
transp_cost = pd.read_csv(path + 'cost.csv', index_col=[0,1]).squeeze("columns")
max_prod = pd.Series([180,200,140,80,180], index = production, name = "max_production")
n_demand = pd.Series([89,95,121,101,116,181], index = distribution, name = "demand") 
frac = 0.75

m = gp.Model('widgets')
x = m.addVars(production, distribution, name = 'prod_ship')
meet_demand = m.addConstrs((gp.quicksum(x[p,d] for p in production) >= n_demand[d] for d in distribution), name = 'meet_demand')
can_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) <= max_prod[p] for p in production), name = 'can_produce')
must_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) >= frac*max_prod[p] for p in production), name = 'must_produce')
m.setObjective(gp.quicksum(transp_cost[i,j]*x[i,j] for i in production for j in distribution), GRB.MINIMIZE) 
m.optimize()

您被告知有一项关于从生产设施运输**小部件**的新政策。现在要求从任何生产设施运往任何配送中心的最小小部件数量需要至少为20。

**问题10-a.** 使用数学符号写出给定新要求后公式的变化。

\begin{align*} 
x_{p,d} \ge 20, \quad \forall p \in P, d \in D
\end{align*}

**问题10-b.** 在下面的单元格中用gurobipy代码写出**问题10-a**的变化（不需要运行它，除非您想复制模型到这里检查）。

In [ ]:
m.addConstrs(x[p,d] >= 20 for p in production for d in distribution)
m.update()

初始小部件模型 $m$ 表示单个时间段（如一周、一个月、一个季度）。假设我们添加了一个时间组件，使用集合 $T = \{0, 1, 2, 3\}$ 表示一年的一个季度。

**问题11-a.** 使用 `addVar()` 或 `addVars()` 在gurobipy中创建一个决策变量，表示在给定时间段内从生产设施运往配送中心的小部件数量。

In [ ]:
x = m.addVars(production, distribution, range(4), name = 'prod_ship')

**问题11-b.** 要引用给定时间 $t$ 之前的时间段，我们可以使用 $t-1$ 作为下标（因为 $T$ 是整数集合），但当 $t = 0$ 时 $t-1$ 不起作用，因为 $-1$ 不在 $T$ 中。在下面的代码中填写 ??? 以表示一组约束，限制生产设施在*连续两个*时间段内的生产量不超过`max_prod[p]`。

In [ ]:
time_prod_limit = m.addConstrs((gp.quicksum(x[p,d,t] + x[p,d,t-1] for d in distribution) <= max_prod[p] for p in production for t in range(4) if t > 0), name = 'time_prod_limit')

In [ ]:
time_prod_limit = m.addConstrs((gp.quicksum(x[p,d,t] + x[p,d,t-1] for d in distribution) <= max_prod[p] for p in production for t in range(4) if t > 0), name = 'time_prod_limit')